In [ ]:
# customary imports:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageOps
import glob
import os
import tqdm
from sklearn.model_selection import StratifiedKFold

tensorflow version: 2.0.0


## 1. Download the white blood cell classification data
The data is hosted online, so we can use the linux command `wget` to download it. If you run into any issues with the data download, please just share your challenges via Slack and we can help sort them out.

In [ ]:
# if this breaks please contact the TAs
!wget -O data.zip https://data.mendeley.com/public-files/datasets/snkd93bnjr/files/2fc38728-2ae7-4a62-a857-032af82334c3/file_downloaded
!unzip /content/data.zip
!unzip /content/PBC_dataset_normal_DIB.zip > /dev/null

In [ ]:
# loading a sample image
sample_image = Image.open("PBC_dataset_normal_DIB/basophil/BA_100102.jpg")
sample_image

In [ ]:
def load_and_crop(image_path, crop_size, normalized=True):
    image = Image.open(image_path).resize([200,200])
    width, height = image.size   # Get dimensions
    left = (width - crop_size)/2
    top = (height - crop_size)/2
    right = (width + crop_size)/2
    bottom = (height + crop_size)/2
    # Crop the center of the image
    image = ImageOps.grayscale(image.crop((left, top, right, bottom)))
    if normalized:
        return np.array(image).astype(np.float32) / 255.0
    else:
        return np.array(image).astype(np.float32)

# code to load all the data, assuming dataset is at PBC_dataset_normal_DIB relative path
cell_types = ['basophil', 'eosinophil', 'erthroblast', 'ig', 'lymphocyte', 'monocyte', 'neutrophil', 'platelet']
cell_inds = np.arange(0, len(cell_types))
x_data = []
y_data = []
for cell_ind in cell_inds:
    all_images = glob.glob(os.path.join('PBC_dataset_normal_DIB', cell_types[cell_ind], '*.jpg'))
    x_data += [load_and_crop(image_path, 128) for image_path in all_images]
    y_data += [cell_ind]*len(all_images)

# adding a fake color channel
x_data = np.array(x_data).reshape(-1, 128, 128, 1)
y_data = np.array(y_data)

folder = StratifiedKFold(5, shuffle=True)
x_indices = np.arange(0, len(x_data))
train_indices, val_indices = folder.split(x_indices, y_data).__next__()
# shuffling
np.random.shuffle(train_indices)

x_train = x_data[train_indices]
y_train = np.eye(len(cell_types))[y_data[train_indices]]

x_val = x_data[val_indices]
y_val = np.eye(len(cell_types))[y_data[val_indices]]

print(x_train.shape, y_train.shape)
print(x_val.shape, y_val.shape)

plt.imshow(x_train[0,:,:,0])
plt.colorbar()

## 2. Define a keras model
You can either use the sequential model class, or the functional model declaration

(a) Please define your model with the following layers:
1. A convolutional layer with a 5x5 kernel and stride of 1
2. A convolutional layer with a 5x5 kernel and stride of 1
3. A pooling layer (Instead of this, you could also increase the stride in the second layer)
4. A convolutional layer with a 5x5 kernel and stride of 1
5. A convolutional layer with a 5x5 kernel and stride of 1
6. A pooling layer (Instead of this, you could also increase the stride in the fifth layer)
7. A Dense layer
8. Output layer of size 8

You are free to choose the sizes, number of channels and activations (i.e., the employed non-linearity) for each of the layers.

(b) Now, please comment out the pooling layer in step 3 and step 6, and instead increase the stride in the appropriate layers to achieve the same down-sampling effect (i.e., to reduce the size of the tensor in the same way as pooling) 

(c) After defining the model, you should define an optimizer and set a learning rate. You also should pick a loss function.

(d) Run the optimization for 10-15 epochs and monitor the training and validation loss and accuracy. After training is done, please plot two graphs, one showing the training and validation losses as two curves within the same plot, and a second graph that shows the the training and validation accuracies as two curves within the same plot. For both plots, please let epoch be the horizontal axis.

At the end of training, you should be able obtain an accuracy better than 80%.

You may refer to the notebook from the TA session or any online TensorFlow resources for guidance.

In [ ]:
# your code here

## 3. How many weight parameters does your network have?
First try calculating this number by hand, and show your work (please type out the multiplications that you are performing to arrive at the final number.) Then, please verify the answer using Keras's autogenerated model summary.

## 4. Visualise filters
You can obtain weights in individual layers by running 
```
your_model_variable.layers[layer_index].get_weights()
```
(a) Plot all convolution kernels (i.e., each set of 5x5 weights) in your first convolutional layer.

(b) What is the variance of final weights in the first convolutional layer? 

(c) Also plot some of the convolutional weights in the second layer. What is the variance of the final weights in the second layer? 


In [ ]:
# your code here

## 5. Try playing with the learning rate
(a) Try to increase and decrease the learning rate and plot the training and validation loss and accuracy curves from part 2(d), for three different values of learning rate that you have tried. 

(b) Please comment on any trends that you can identify between how the plots change as a function of learning rate. Specifically, what happens to the slopes of the training loss and accuracy as a function of learning rate?

In [ ]:
# your code here

## 6. Adding Batch Norm
Fix a value of the learning rate and try adding Batch Normalization after layers 2 and 5. Does it improve the performance of your model? Explain briefly.

In [ ]:
# your code here

## 7. Data Augmetation
Now, instead of giving the dataset directly to the network, augment it first using:
```
keras.preprocessing.image.ImageDataGenerator
```
Specifically, use vertical and horizontal flips and 20 degrees rotation. We also want to normalise the data. Feel free to consult the documentation for this function.
What effect does this have on your model?

In [ ]:
# your code here

## 8. Custom layers
In one of the TA sessions, we briefly went over how to implement a custom layer -- specifically, we re-implemented the Dense (or fully-connected) layer. In this part, we will get more practice implementing custom layers.

Please reimplement a simplified version of `tf.keras.layers.Conv2D` using `tf.nn.conv2d`: https://www.tensorflow.org/api_docs/python/tf/nn/conv2d

Note carefully the difference between these two tf constructs. In particular, `tf.keras.layers.Conv2D` is a high-level implementation of the 2D convolution that defines all the parameters under the hood, while `tf.nn.conv2d` requires you to define your own convolutional kernels via `tf.Variable` (you may also use the `add_weight` function if you desire). 
- Your implementation should also include a bias variable, consistent with the default behavior of `tf.keras.layers.Conv2D`.
- Your constructor should accept 4 parameters: filters, kernel_size, strides, and activation.
- You may hard-code the padding as `'SAME'`
- You do not have to implement the string shortcuts for the activations (e.g., if your code handles `tf.nn.relu` but not `'relu'`, that's okay).
- For simplicity, you may assume that kernel_size and strides are integers (i.e., as opposed to lists).
- Initialize all weights using the standard normal distribution.

Note that since you only have to deal with the above 5 input arguments, your implementation will not be as sophisticated as `tf.keras.layers.Conv2D`, which contains many other input arguments. Rather, the point of this exercise is for you to get a better understanding of what tf is doing under the hood so that you are not just blindly using their high-level functions.

Feel free to refer to the notebook from the TA session or any online tf documentation, though please do not copy the source code from tf's native implementation of Conv2D.

After you're done, repeat the CNN defined above, substituting all instances of `tf.keras.layers.Conv2D` with your implementation, and run for 10 epochs. It's okay if you don't get the same accuracy, but it should still improve. Also print out the `.summary()` command to ensure that the number of parameters is the same as before.

In [ ]:
class Conv2D(tf.keras.layers.Layer):
  def __init__(self, filters, kernel_size, strides, activation):
    super().__init__()
    pass
  
  def build(self, input_shape):
    # expect the input_shape to be (batch_size, height, width, filters_previous)
    pass
  
  def call(self, input):
    pass

In [ ]:
# copy and paste an earlier training script, and replace the native conv2Ds with yours

##** Bonus question: Custom layer for Fourier filtering

Note: this problem requires some careful bug-checking

Now, we will implement a custom layer that doesn't exist in keras -- Fourier filtering. Your layer should apply the 2D Fourier transform (`tf.signal.fft2d`) to each channel of the input, multiply element-wise by an optimizable mask (a different one for each channel), apply the 2D inverse Fourier transform (`tf.signal.ifft2d`), and then take the absolute value. Note: 
- You will have to use the tf versions of all operations, NOT the numpy versions. 
- The fft2d operations in tensorflow are done on the LAST two dimensions, which is at odds with the default dimension ordering of CNNs. Thus, you will need to use `tf.transpose` on the input and then transpose back after the filtering operation.
- Use dtype `tf.complex64`, which is basically a combination of two `tf.float32`s. You will have to explicitly cast between these two data types, because the input/output will be `tf.float32`, but intermediate steps will be `tf.complex64`.

Initialize your optimizable Fourier masks using a binary circular mask (1's inside the circle, 0's outside), with a radius given by 1/4 of the square image dimension (you can round if not divisible by 4). 

After defining this custom layer, copy your previously defined CNN above and insert this new layer as the first layer. To verify that your layer is working correctly, plot some example outputs of the first layer.

In [ ]:
class FourierFilter(tf.keras.layers.Layer):
  def __init__(self):
    super().__init__()
  
  def build(self, input_shape):
    # expect the input_shape to be (batch_size, height, width, filters_previous)
    pass
  
  def call(self, input):
    pass